# LoKI: data reduction from Larmor detector test

This notebook shows how to use the `LokiAtLarmorWorkflow` to reduce data acquired during the 2022 LoKI detector test at Larmor.

We will set up a standard workflow that computes a one-dimensional $I(Q)$ from the raw data,
and reuse the workflow for 4 different samples:

- ISIS polymer
- AgBeh
- Porous silica
- deut-SDS

In [ ]:
import scipp as sc
import plopp as pp
from ess import sans
from ess import loki
import ess.loki.data  # noqa: F401
from ess.sans.types import *

## Create and configure the workflow

We begin by creating the Loki workflow object:

In [ ]:
workflow = loki.LokiAtLarmorWorkflow()

# Load masks from files
workflow = sans.with_pixel_mask_filenames(
    workflow, masks=loki.data.loki_tutorial_mask_filenames()
)
# Set detector bank name: in this case there is only one bank
workflow[NeXusDetectorName] = "larmor_detector"

# Wavelength and Q binning parameters
workflow[WavelengthBins] = sc.linspace("wavelength", 1.0, 13.0, 201, unit="angstrom")
workflow[QBins] = sc.linspace(dim="Q", start=0.01, stop=0.3, num=101, unit="1/angstrom")

# Other parameters
workflow[CorrectForGravity] = True
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound
workflow[ReturnEvents] = False

### File common to all runs

We set files that are common to all sample runs, namely the background, empty beam and direct beam files:

In [ ]:
workflow[Filename[BackgroundRun]] = loki.data.loki_tutorial_background_run_60393()
workflow[Filename[TransmissionRun[BackgroundRun]]] = loki.data.loki_tutorial_run_60392()
workflow[Filename[EmptyBeamRun]] = loki.data.loki_tutorial_run_60392()
workflow[DirectBeamFilename] = loki.data.loki_tutorial_direct_beam_all_pixels()

## ISIS Polymer sample

We configure the files to load

In [ ]:
workflow[Filename[SampleRun]] = loki.data.loki_tutorial_isis_polymer_sample_run()
workflow[Filename[TransmissionRun[SampleRun]]] = (
    loki.data.loki_tutorial_isis_polymer_transmission_run()
)

### Computing the beam center

We use a center-of-mass approach to find the beam center:

In [ ]:
center = sans.beam_center_from_center_of_mass(workflow)
center

and set that value in our workflow

In [ ]:
workflow[BeamCenter] = center

### Visualizing the workflow

It is always a good idea to visualize the workflow graph, as a sanity check.

In [ ]:
workflow.visualize(BackgroundSubtractedIofQ, compact=True, graph_attr={"rankdir": "LR"})

### Intermediate result: transmission fraction

In [ ]:
tf = workflow.compute(TransmissionFraction[SampleRun])
tf.plot()

### Compute I(Q)

In [ ]:
da = workflow.compute(BackgroundSubtractedIofQ)
da.plot(norm="log", title="ISIS polymer")

### Wavelength bands

In [ ]:
workflow[WavelengthBands] = sc.linspace("wavelength", 1.0, 13.0, 11, unit="angstrom")
da_bands = workflow.compute(BackgroundSubtractedIofQ)
da_bands

In [ ]:
pp.plot(sc.collapse(da_bands, keep="Q"), norm="log", title="ISIS polymer")

## AgBeh sample

In [ ]:
# Reset wavelength bands
workflow[WavelengthBands] = None

workflow[Filename[SampleRun]] = loki.data.loki_tutorial_agbeh_sample_run()
workflow[Filename[TransmissionRun[SampleRun]]] = (
    loki.data.loki_tutorial_agbeh_transmission_run()
)

In [ ]:
# Beam center
workflow[BeamCenter] = sans.beam_center_from_center_of_mass(workflow)

In [ ]:
# I(Q)
da = workflow.compute(BackgroundSubtractedIofQ)
da.plot(norm="log", title="AgBeh sample")

## Porous Silica

In [ ]:
workflow[Filename[SampleRun]] = loki.data.loki_tutorial_porous_silica_sample_run()
workflow[Filename[TransmissionRun[SampleRun]]] = (
    loki.data.loki_tutorial_porous_silica_transmission_run()
)

# Beam center
workflow[BeamCenter] = sans.beam_center_from_center_of_mass(workflow)

# I(Q)
da = workflow.compute(BackgroundSubtractedIofQ)
da.plot(norm="log", title="Porous silica")

## Deut-SDS

In [ ]:
workflow[Filename[SampleRun]] = loki.data.loki_tutorial_deut_sds_sample_run()
workflow[Filename[TransmissionRun[SampleRun]]] = (
    loki.data.loki_tutorial_deut_sds_transmission_run()
)

# Beam center
workflow[BeamCenter] = sans.beam_center_from_center_of_mass(workflow)

# I(Q)
da = workflow.compute(BackgroundSubtractedIofQ)
da.plot(norm="log", title="deut-SDS")